In [3]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
import mediapipe as mp
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model



mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def calculate_distance(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def extract_pose_features(video_path, model, scaler_X, encoder):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        alto_original, ancho_original = frame.shape[:2]
        nuevo_alto = 1100  # Nueva altura deseada
        nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
        frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            keypoints = [(lm.x, lm.y, lm.z) for lm in landmarks]

            # Extract relevant angles and distances
            left_shoulder = keypoints[11]
            right_shoulder = keypoints[12]
            left_elbow = keypoints[13]
            right_elbow = keypoints[14]
            left_wrist = keypoints[15]
            right_wrist = keypoints[16]
            left_hip = keypoints[23]
            right_hip = keypoints[24]
            left_knee = keypoints[25]
            right_knee = keypoints[26]
            left_ankle = keypoints[27]
            right_ankle = keypoints[28]

            # Example angles
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
            left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
            right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
            left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
            right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
            
            # Example distances
            wrist_distance = calculate_distance(left_wrist, right_wrist)
            ankle_distance = calculate_distance(left_ankle, right_ankle)
            shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
            hip_distance = calculate_distance(left_hip, right_hip)

            features = np.array(keypoints).flatten().tolist()

            X = scaler_X.transform(np.array(features).reshape(1, -1))
            predictions = model.predict(X)

            # Separar las predicciones
            predicciones_continuas = predictions[0]
            prediccion_clase = encoder.inverse_transform(np.argmax(predictions[1], axis=1))

            features_video = [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle, wrist_distance, ankle_distance, shoulder_distance, hip_distance]

            # Comparar características del video con las predichas
            porcentaje_diferencia = 0.15  # Porcentaje de diferencia permitido
            errores = np.where(np.abs(np.array(features_video) - predicciones_continuas) > porcentaje_diferencia)[0]
            errores = list(set(errores))

            caracteristicas = ["Angulo del codo izquierdo", "Angulo del codo derecho", "Angulo del hombro izquierdo", "Angulo del hombro derecho", "Angulo de la rodilla izquierda", "Angulo de la rodilla derecha", "Distancia entre munecas", "Distancia entre tobillos", "Distancia entre hombros", "Distancia entre caderas"]


            # Dibujar la pose
            annotated_image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                annotated_image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
            )

            clase_predicha = prediccion_clase[0]
            porcentaje_prediccion = np.max(predictions[1]) * 100

            # Mostrar la predicción de clase
            texto_clasificacion = f'Clasificacion: {clase_predicha} - Probabilidad: {porcentaje_prediccion:.2f}%'
            cv2.putText(annotated_image, texto_clasificacion, (50, 220), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)


            if len(errores) > 0:
                for i, error in enumerate(errores):
                    desplazamiento_vertical = 240 + (i * 30)  # Incrementa la posición vertical para cada error adicional
                    cv2.putText(annotated_image, f'Error en : {caracteristicas[error]}', (50, desplazamiento_vertical), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            
            cv2.imshow('Frame', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))

            # Escuchar tecla 'q' para salir
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()



# Cargar el modelo y las clases
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_dataset = os.path.join(ruta_raiz, 'Data Set') 
ruta_video = os.path.join(ruta_dataset, 'test_video.mp4')
ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual,'datos','scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')


# Cargar el modelo y scaler
model = load_model(ruta_modelo)
scaler_X = joblib.load(ruta_scaler)

classes = np.load(ruta_npy, allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes

# Realizar la predicción
extract_pose_features(ruta_video,model,scaler_X,encoder)



c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 